## Apiux & SII: calculo de indice de familiariedad en  personas juridicas.
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [27]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/DatoOrigen/lr-629/UtilBajo/intermedia/familiaridad") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

In [12]:
spark.sql("show databases").show()

+--------------------+
|           namespace|
+--------------------+
|            apa_temp|
|             default|
|                  dw|
|            dwbgdata|
|   fraude_receptores|
|identificacion_ac...|
|  information_schema|
|iva_sobredeclaracion|
|  iva_subdeclaracion|
|              libaar|
|              libsas|
|              libsdf|
|rentaempresa_simi...|
|                 sdj|
|                 sys|
|                 tmp|
|            utilbajo|
+--------------------+



In [13]:
spark.sql("use utilbajo")

DataFrame[]

In [14]:
spark.sql("show databases").show()

+--------------------+
|           namespace|
+--------------------+
|            apa_temp|
|             default|
|                  dw|
|            dwbgdata|
|   fraude_receptores|
|identificacion_ac...|
|  information_schema|
|iva_sobredeclaracion|
|  iva_subdeclaracion|
|              libaar|
|              libsas|
|              libsdf|
|rentaempresa_simi...|
|                 sdj|
|                 sys|
|                 tmp|
|            utilbajo|
+--------------------+



In [15]:
#spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatoOrigen/lr-629/UtilBajo/intermedia/familiaridad").show()


## Carga de relaciones societarias(depurada)

In [22]:
df = spark.read.options(header=True, inferSchema=True, delimiter=",").csv("/home/cdsw/data/processed/sociedades_participacion_capital_nozero.csv")
df.createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad order by RUT_SOCIEDAD asc").show()

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|+++1gCGSMhlEHLJDQ...|jBIDEmi35uF/bAHr2...|             100.0|                100.0|
|+++40fb1kG88W0AEf...|bmXOLtRFjJPrLjKzQ...|             100.0|                100.0|
|+++aS3F42zAfvw7QX...|WdafNmzolsFbUp2HA...|             100.0|                100.0|
|++/7Y0IzKUdH2J8C/...|41WRUY90glpRrq9XI...|             100.0|                100.0|
|++/p+MzwTKR1XVz6S...|+bJxBrWdS6tCHMPtN...|             100.0|                100.0|
|++00DcdTZAhzp008i...|aL6LEEk8P6vB8PyHu...|             100.0|                100.0|
|++0Cw0fQKv+4UABY2...|eJvYBKdvqhJ6I1vK5...|             100.0|                100.0|
|++0PaA9Gk1ZMElZZL...|Y20GxtSITZxJGWolh...|              50.0|                 50.0|
|++0PaA9Gk1ZMElZZL...|VkY6uutbGR+7iTLUS...|              50.0|   

Veamos un ejemplo de la composicion de socios para una sociedad sospechosa en composicion

In [21]:
spark.sql("select * from sociedad where RUT_SOCIEDAD LIKE 'G8Q3ZKZV1VBYw0fT%' or RUT_SOCIO LIKE 'G8Q3ZKZV1VBYw0fT%'").show()
#spark.sql("select *  from libsdf.jab_soc_2023_inom where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|LVBWLDHGQBkRZPe80...|G8Q3ZKZV1VBYw0fTh...|             24.84|                24.84|
|G8Q3ZKZV1VBYw0fTh...|XeWWsDI+O4GpcHzh+...|              25.0|                  1.0|
|G8Q3ZKZV1VBYw0fTh...|eMvqyYgN6pwUQTGfr...|              25.0|                  1.0|
|G8Q3ZKZV1VBYw0fTh...|tcWo0xZnJksYr6Uye...|              25.0|                  1.0|
|HixT5exYVJ7YqDGR7...|G8Q3ZKZV1VBYw0fTh...|             17.32|                17.32|
|G8Q3ZKZV1VBYw0fTh...|/WglMrVtSfq9fsB/N...|              25.0|                  1.0|
|pMAyyWg4gct9P4O8x...|G8Q3ZKZV1VBYw0fTh...|             18.32|                18.32|
|q1rQbXJ8m7nQEni/h...|G8Q3ZKZV1VBYw0fTh...|             17.32|                17.32|
+--------------------+--------------------+------------------+---

In [30]:

spark.sql('select * from libsdf.jab_soc_2023_inom').createOrReplaceTempView("socios")
spark.sql('select * from socios')

DataFrame[RUT_SOCIEDAD: string, RUT_SOCIO: string, PORCENTAJE_CAPITAL: double, PORCENTAJE_UTILIDADES: double, PERI_AGNO_MES_TRIBUTARIO: int, FUENTE: string]

In [23]:
data_test=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/LibSDF/GE_APIUX_ARFI_E").createOrReplaceTempView("grupos_conocidos")
spark.sql('select PARU_RUT_E,COM from grupos_conocidos group by PARU_RUT_E, COM order by PARU_RUT_E asc').createOrReplaceTempView("grupos_conocidos")
spark.sql('select count(*) from grupos_conocidos left join sociedad on grupos_conocidos.PARU_RUT_E=sociedad.RUT_SOCIEDAD where RUT_SOCIEDAD is not null').show()

+--------------------+----+------------+---------+------------------+---------------------+
|          PARU_RUT_E| COM|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+----+------------+---------+------------------+---------------------+
|++ggVpHHILx+9SGu8...|1141|        null|     null|              null|                 null|
|+1FNxAxXZVPFSdmBl...|1920|        null|     null|              null|                 null|
|++qoXOhoawzvRog9E...| 999|        null|     null|              null|                 null|
|+++4/3jzUwtDPSSo3...| 460|        null|     null|              null|                 null|
|+0f/GM34xojh+tn7A...| 321|        null|     null|              null|                 null|
|+/bsZuw7/ADsTou53...|2202|        null|     null|              null|                 null|
|+0dURcPCBYLGVKmIt...| 614|        null|     null|              null|                 null|
|++7yFnLR2jk0Wr0e8...|2478|        null|     null|              null|           

In [31]:
spark.sql('select count(*) from grupos_conocidos left join socios on grupos_conocidos.PARU_RUT_E=socios.RUT_SOCIEDAD where RUT_SOCIEDAD is not null').show()

24/03/28 17:57:40 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


+--------+
|count(1)|
+--------+
|   63621|
+--------+



## Tablas temporales previo a la ejecucion

In [9]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO from sociedad order by RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD_AUX ,RUT_SOCIO as RUT_SOCIO_AUX from sociedad order by RUT_SOCIEDAD asc").createOrReplaceTempView("aux")
#spark.sql("select RUT_SOCIEDAD, COUNT(*) AS F from sociedad group by RUT_SOCIEDAD ORDER BY F DESC").show()



## Calculo de arbol de socios naturales

In [10]:
for a in range (1,10):
    spark.sql("select * from sociedad left join aux on sociedad.RUT_SOCIO=aux.RUT_SOCIEDAD_AUX ").createOrReplaceTempView("sociedad")
    spark.sql("select * from sociedad order by RUT_SOCIEDAD_AUX desc").createOrReplaceTempView("sociedad")
    #spark.sql("select * from sociedad").show()
    spark.sql("select RUT_SOCIEDAD, CASE WHEN RUT_SOCIEDAD_AUX is null then RUT_SOCIO else RUT_SOCIO_AUX END AS RUT_SOCIO from sociedad  order by RUT_SOCIEDAD_AUX desc").createOrReplaceTempView("sociedad")
    #spark.sql("select * from sociedad").show()
    #spark.sql("select RUT_SOCIEDAD, COUNT(*) as d from sociedad GROUP BY RUT_SOCIEDAD order by d desc ").show()

In [11]:
oscuridad=spark.sql("select * from libsdf.jab_materia_inom")
oscuridad.createOrReplaceTempView("oscuridad")
spark.sql("select RUT_SOCIEDAD, CONT_RUT from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT ").createOrReplaceTempView("socios_final")

## Sociedades por persona natural

Tambien obtendremos un output que nos permita establecer cuanta sociedades esta relacionado con cada uno de las personas naturales relacionadas.

In [12]:
sociedades_por_socio=spark.sql("select CONT_RUT, count(RUT_SOCIEDAD) as SOCIEDADES_RELACIONADAS from socios_final where CONT_RUT is not null group by CONT_RUT order by SOCIEDADES_RELACIONADAS DESC").toPandas()
sociedades_por_socio.to_csv('/home/cdsw/artefactos/Familiaridad/sociedades_por_socio.csv', index=False)

In [13]:
spark.sql("select RUT_SOCIEDAD, COUNT(*) AS NONULOS from socios_final where CONT_RUT is not null group by RUT_SOCIEDAD ").createOrReplaceTempView("nonulos")
spark.sql("select RUT_SOCIEDAD, COUNT(*) AS TOTAL from socios_final group by RUT_SOCIEDAD ").createOrReplaceTempView("total")

spark.sql("select  total.RUT_SOCIEDAD as RUT_SOCIEDAD, TOTAL, NONULOS from total left join nonulos on total.RUT_SOCIEDAD=nonulos.RUT_SOCIEDAD").createOrReplaceTempView("final")

In [14]:
spark.sql("select * from final").show()

+--------------------+-----+-------+
|        RUT_SOCIEDAD|TOTAL|NONULOS|
+--------------------+-----+-------+
|/iaWdrAtFePvm67ix...| 4831|   4444|
|0/u1ahHIS1sVRlQ6K...|   14|     11|
|50jo0+NXwtdYFAila...|    2|      2|
|5lyqqCo43kjf4EntX...|    2|      2|
|6EP3XO6W7pdCljd/8...|    1|      1|
|6EYmB54CdYIoQWakH...| 1124|   1042|
|B5F+dlQW541SF7kKm...| 3536|   3342|
|EIZ7wxHir6Lzswl7V...|    2|      2|
|Kpoq2KViEnP9fCv5I...|    3|      3|
|PEi/p+2zp2fFFaxBE...|    3|      3|
|PH98Pjz/e71FkUq0Y...|    2|      2|
|WOIhov6M/ymNM3aI1...|    2|      1|
|Z1viwZ0tUtu4l+4Zz...|    5|      5|
|iG6Zx7RL2DRkTOf94...|    2|      2|
|jBjQYicsPaokMPI4K...|  247|    231|
|mtmCVDI5lLdIWNPWF...|  208|    189|
|nAUfOM2fyYbJ/fyK0...|   37|     37|
|nTxOYTX2RpgxeCx+S...|   17|     15|
|s4Gs6FthB6MSF7VEw...|    2|      2|
|uGs5LUeIhI1PrMh7z...| 7591|   6597|
+--------------------+-----+-------+
only showing top 20 rows



Trabajaremos con las sociedades que en su arbol relacional contienen todas sus personas naturales completas y con un numero total de personas naturales conectadas menor que 100.

In [15]:
#spark.sql("select socios_final.RUT_SOCIEDAD, CONT_RUT as RUT_SOCIO from socios_final left join final on socios_final.RUT_SOCIEDAD=final.RUT_SOCIEDAD where TOTAL=NONULOS AND TOTAL<100 ORDER BY TOTAL desc").createOrReplaceTempView("final")

In [16]:
spark.sql("select socios_final.RUT_SOCIEDAD, CONT_RUT as RUT_SOCIO from socios_final left join final on socios_final.RUT_SOCIEDAD=final.RUT_SOCIEDAD where TOTAL=NONULOS AND TOTAL<100 ORDER BY TOTAL desc").createOrReplaceTempView("final")

## Combinatoria de pool de socios

A continuacion, se hace un auto join con el fin de obtener todas las combinaciones posibles entre socios diferentes para comparar con las relaciones familiares. Luego de ello se concatenan los socios para establecer un codigo especifico para cada combinacion de ello.

In [17]:
spark.sql("select t1.RUT_SOCIEDAD, t1.RUT_SOCIO AS RUT_SOCIO_1, t2.RUT_SOCIO AS RUT_SOCIO_2 FROM final AS t1 JOIN final AS t2 ON t1.RUT_SOCIEDAD = t2.RUT_SOCIEDAD  WHERE t1.RUT_SOCIO<>t2.RUT_SOCIO").createOrReplaceTempView("final")
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO_1,RUT_SOCIO_2,RUT_SOCIO_1||RUT_SOCIO_2 as key from final").createOrReplaceTempView("final")

In [18]:
spark.sql('select * from final').show()

+--------------------+--------------------+--------------------+--------------------+
|        RUT_SOCIEDAD|         RUT_SOCIO_1|         RUT_SOCIO_2|                 key|
+--------------------+--------------------+--------------------+--------------------+
|++nemOqjdfZAI1/ba...|/qYdWqtDBGWY8SO1l...|tJ+L1N5qx9qHOF2Ur...|/qYdWqtDBGWY8SO1l...|
|++nemOqjdfZAI1/ba...|tJ+L1N5qx9qHOF2Ur...|/qYdWqtDBGWY8SO1l...|tJ+L1N5qx9qHOF2Ur...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|+RRIvh4eJEZAJ/SFu...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|+RRIvh4eJEZAJ/SFu...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|+RRIvh4eJEZAJ/SFu...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|ucY0yK5VSzWN5VHzQ...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|wKZ5OnarYzxbH1Sdl...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|wKZ5OnarYzxbH1Sdl...|gvBiz2qI1ieZwbgp+...|
|+/5oc4ItzHvxHH5xZ...|gvBiz2qI1ieZwbgp+...|wKZ5OnarYzx

## Exploracion, limpieza y ampliacion de  data de relaciones familiares

A continuacion se obtiene el archivo de relaciones familiares que permitira obtener las relaciones entre socios y comparar con los datos de composicion de sociedades. En este archivo hay diferentes relaciones familiares. Veamos cuales hay:

In [19]:
spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/LibSDF/REL_FAMILIARES_AARI_EH").createOrReplaceTempView("familiar")
spark.sql("SELECT TIPO_RELACION, count(*) FROM familiar group by TIPO_RELACION").show()
spark.sql("SELECT count(*) from familiar").show()

+-------------+--------+
|TIPO_RELACION|count(1)|
+-------------+--------+
|      COPADRE|12451621|
|  NUERA/YERNO| 3892708|
|       ABUELO| 8007965|
|       SUEGRO| 3892529|
|          TIO|27666477|
|        MADRE|12957718|
|       ABUELA| 8084065|
|        PRIMO|39579326|
|      SOBRINO|27666398|
|        NIETO|16090759|
|      CONYUGE| 6416639|
|       CU�ADO|13117996|
|        PADRE|12485678|
|      HERMANO|32675727|
|         HIJO|25440172|
|TIPO_RELACION|       2|
+-------------+--------+

+---------+
| count(1)|
+---------+
|250425780|
+---------+



A continuacion se toman en cuenta solo una vez los datos repetidos y no consideramos las relaciones donde CONT_RUT sea igual a RUT_FAM.

In [20]:
spark.sql("select CONT_RUT,RUT_FAM,COUNT(*) as c from familiar where CONT_RUT!=RUT_FAM group by CONT_RUT,RUT_FAM order by c desc ").createOrReplaceTempView("familiar")
spark.sql("select CONT_RUT,RUT_FAM from familiar").createOrReplaceTempView("familiar")

In [21]:
spark.sql('select * from familiar').show()

+--------------------+--------------------+
|            CONT_RUT|             RUT_FAM|
+--------------------+--------------------+
|l6v3yfr8MHTwngudl...|HGiMRARQ7R/I1u2IQ...|
|of4XHhwnFXfMBsRyF...|umanTm3FtRQuZKUcr...|
|61Ezgpa4hNfeBaBJ9...|5XG2JhO5Mo4WS0Bgj...|
|agUVtBHzQGw/dRd1t...|Wbp4c+jGXDCrk28G/...|
|gOSQk3bkLgrE0aA9N...|m5/69JR6M6YqdpUKQ...|
|m5/69JR6M6YqdpUKQ...|gOSQk3bkLgrE0aA9N...|
|Wbp4c+jGXDCrk28G/...|agUVtBHzQGw/dRd1t...|
|umanTm3FtRQuZKUcr...|of4XHhwnFXfMBsRyF...|
|C8WccdE039hep4E86...|UAxTECWO51t/YicmY...|
|5XG2JhO5Mo4WS0Bgj...|61Ezgpa4hNfeBaBJ9...|
|aJBbKP9QQEURgg5VL...|9X0JUZKFTTcbzFXGu...|
|C8WccdE039hep4E86...|ORiNV1owHosOBw+9s...|
|ORiNV1owHosOBw+9s...|C8WccdE039hep4E86...|
|9X0JUZKFTTcbzFXGu...|aJBbKP9QQEURgg5VL...|
|HGiMRARQ7R/I1u2IQ...|l6v3yfr8MHTwngudl...|
|UAxTECWO51t/YicmY...|C8WccdE039hep4E86...|
|jgt20Hrk9J3YswZY0...|2wbmrHURHRzwv3LjU...|
|6yNNEou4iSWtQX5CU...|EPGFC1V5Qvcou0gSw...|
|emHeeIWtYDvTGeYmx...|qNDhR6Hrn1h211sLl...|
|xi9P/ly5TcFL0Go4z...|18DZoCmtIt

Duplicamos la data e invertimos las relaciones porque la familiariedad es bidireccional.

In [22]:
spark.sql("select CONT_RUT as RUT_FAM,RUT_FAM as CONT_RUT from familiar").createOrReplaceTempView("familiar2")
spark.sql("SELECT * FROM familiar UNION ALL SELECT * FROM familiar2").createOrReplaceTempView("familiar")

In [23]:
spark.sql('select * from familiar').show()

+--------------------+--------------------+
|            CONT_RUT|             RUT_FAM|
+--------------------+--------------------+
|l6v3yfr8MHTwngudl...|HGiMRARQ7R/I1u2IQ...|
|61Ezgpa4hNfeBaBJ9...|5XG2JhO5Mo4WS0Bgj...|
|UAxTECWO51t/YicmY...|C8WccdE039hep4E86...|
|C8WccdE039hep4E86...|ORiNV1owHosOBw+9s...|
|HGiMRARQ7R/I1u2IQ...|l6v3yfr8MHTwngudl...|
|m5/69JR6M6YqdpUKQ...|gOSQk3bkLgrE0aA9N...|
|9X0JUZKFTTcbzFXGu...|aJBbKP9QQEURgg5VL...|
|umanTm3FtRQuZKUcr...|of4XHhwnFXfMBsRyF...|
|ORiNV1owHosOBw+9s...|C8WccdE039hep4E86...|
|5XG2JhO5Mo4WS0Bgj...|61Ezgpa4hNfeBaBJ9...|
|agUVtBHzQGw/dRd1t...|Wbp4c+jGXDCrk28G/...|
|of4XHhwnFXfMBsRyF...|umanTm3FtRQuZKUcr...|
|Wbp4c+jGXDCrk28G/...|agUVtBHzQGw/dRd1t...|
|C8WccdE039hep4E86...|UAxTECWO51t/YicmY...|
|gOSQk3bkLgrE0aA9N...|m5/69JR6M6YqdpUKQ...|
|aJBbKP9QQEURgg5VL...|9X0JUZKFTTcbzFXGu...|
|x8R5Rxw6vgn+hfWtk...|geKFexZDjD+KniyPe...|
|0UxFLow+CHEmFxSZv...|yfTbBMjnH1T+mkrWX...|
|IOxVuUEBL7ldwDEk5...|NH0JLiIEU+DYM4hCh...|
|ib47oqN+ReVIV7ipX...|3sdgCy2RXa

In [24]:
spark.sql("select CONT_RUT, RUT_FAM,count(*) AS C from familiar GROUP BY CONT_RUT, RUT_FAM ORDER BY C DESC").createOrReplaceTempView("familiar")
spark.sql("select  CONT_RUT || RUT_FAM  as key from familiar").createOrReplaceTempView("familiar")
spark.sql("select  * from familiar").show()


24/03/28 01:24:58 WARN WatcherWebSocketListener: Exec Failure javax.net.ssl.SSLException Connection reset
24/03/28 01:24:59 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 129645799 (129646257)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onStatus(AbstractWatchManager.java:265)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onMessage(AbstractWatchManager.java:249)
	at io.fabric8.kubernetes.client.dsl.internal.WatcherWebSocketListener.onMessage(WatcherWebSocketListener.java:93)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okht

+--------------------+
|                 key|
+--------------------+
|+++1iAkZoVFs4IYW1...|
|+++CLj7Nrfzd8XMK3...|
|+++CpnJr+3bIlrHhU...|
|+++e2oIABHElx6zNU...|
|++/CyX6OXvsJdI1OW...|
|++/SwvlV41jDvMXpB...|
|++0Rw2W/nMLfZH7Pb...|
|++0e2Ci9b007rBquz...|
|++0pECiRYm13q80ro...|
|++0rJxbsXtRSk3MAC...|
|++1+u3dQTgCoAdhmP...|
|++10BoDkN5EhH5kcS...|
|++12vwvM+46PtJOYp...|
|++19b0STpoaFhLQ0e...|
|++1AJ3MTrt2S5hUiJ...|
|++1Ijxv8EYognqql4...|
|++1LGWOxLvzqIKGvJ...|
|++1eXvnNa+kox3Iax...|
|++1kgkiRX96/VDfB/...|
|++1lrc6WeHwSb3DK2...|
+--------------------+
only showing top 20 rows



## Cruce de relaciones familiares calculadas con pool de socios por sociedad

In [25]:
spark.sql("select RUT_SOCIEDAD,RUT_SOCIO_1, RUT_SOCIO_2, familiar.key as FAMILIAR_KEY from final left join familiar on final.key= familiar.key order by RUT_SOCIEDAD").createOrReplaceTempView("relaciones")
spark.sql("select * from relaciones").show()

+--------------------+--------------------+--------------------+------------+
|        RUT_SOCIEDAD|         RUT_SOCIO_1|         RUT_SOCIO_2|FAMILIAR_KEY|
+--------------------+--------------------+--------------------+------------+
|++0PaA9Gk1ZMElZZL...|Y20GxtSITZxJGWolh...|VkY6uutbGR+7iTLUS...|        null|
|++0PaA9Gk1ZMElZZL...|VkY6uutbGR+7iTLUS...|Y20GxtSITZxJGWolh...|        null|
|++1jdKtXA14GodsF2...|/VVBxeWcPKI3JZxh1...|BNuqVX553Rgbk6Y4p...|        null|
|++1jdKtXA14GodsF2...|BNuqVX553Rgbk6Y4p...|/VVBxeWcPKI3JZxh1...|        null|
|++1zBpa3uYVD9MM8r...|FxBSqC1xZi3jQG8ii...|vxQVwz2Mb6H3evEDj...|        null|
|++1zBpa3uYVD9MM8r...|vxQVwz2Mb6H3evEDj...|FxBSqC1xZi3jQG8ii...|        null|
|++2HU9Dm1tPPFCT0i...|3b5vEKiAaZ+phVPw+...|9s5GGH0n6Ng8nMFn9...|        null|
|++2HU9Dm1tPPFCT0i...|lV2cZG2d/PVbIHB9o...|3b5vEKiAaZ+phVPw+...|        null|
|++2HU9Dm1tPPFCT0i...|9s5GGH0n6Ng8nMFn9...|3b5vEKiAaZ+phVPw+...|        null|
|++2HU9Dm1tPPFCT0i...|3b5vEKiAaZ+phVPw+...|lV2cZG2d/PVbIHB9o...|

24/03/28 02:06:16 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /10.244.74.14:59620 is closed
24/03/28 02:06:16 WARN BlockManagerMasterEndpoint: Error trying to remove shuffle 86 from block manager BlockManagerId(26, 10.244.74.14, 40553, None)
java.io.IOException: Connection from /10.244.74.14:59620 closed
	at org.apache.spark.network.client.TransportResponseHandler.channelInactive(TransportResponseHandler.java:147)
	at org.apache.spark.network.server.TransportChannelHandler.channelInactive(TransportChannelHandler.java:117)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:262)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:248)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelInactive(AbstractChannelHandlerContext.java:241)
	at io.netty.channel.ChannelInboundHandlerAdapter.channelInactive(ChannelInboundHandlerAdapt

Lo que haremos a continuacion es obtener los valores unicos de socios que si tienen relaciones, que son los distintos valores de RUT_SOCIO que tienen FAMILIAR_KEY no nulo.

In [ ]:
spark.sql("select RUT_SOCIEDAD,COUNT(DISTINCT(RUT_SOCIO_1)) AS FAMILIARES from relaciones WHERE FAMILIAR_KEY IS NOT NULL GROUP BY RUT_SOCIEDAD").createOrReplaceTempView("socios_familia")
spark.sql("select socios_familia.RUT_SOCIEDAD, FAMILIARES, TOTAL  from socios_familia right join total on socios_familia.RUT_SOCIEDAD=total.RUT_SOCIEDAD").createOrReplaceTempView("socios_familia")


In [ ]:
## Calculo final de metrica asociadaa familiaridad. Explicacion y 

In [ ]:
spark.sql("select  RUT_SOCIEDAD, FAMILIARES, TOTAL, FAMILIARES/TOTAL*100 as TASA_FAMILIARIDAD from socios_familia").toPandas().to_csv('artefactos/Familiaridad/familiaridad.csv', index=False)

In [ ]:
#a.write.csv("familiar_output.csv")